
# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shap
import pickle

from PositionalModel import PositionalModel
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import LocalOutlierFactor
import copy

from config import *
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings("ignore", message="Empty dataset at worker")

# Loading data
Data consists of all PL players logs for 2021-22 and 2022-23 seasons\
A single log is just a summary of player performance in a particular match

In [3]:
# loading csv
df_original = pd.read_csv('data/previous/FPL_logs.csv')

df = df_original.copy()

In [4]:
df = df.drop(columns=["FPL_GW",
    "Min_points",
    "Clean_name",
    "FPL_xA",
    "FPL_xGC",
    "FPL_xG",
    "FPL_xGI"]
    )
df = df.dropna()

In [5]:
df.columns

Index(['Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name', 'Start', 'Pos',
       'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR',
       'Touches', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA', 'SCA', 'GCA',
       'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries', 'PrgC', 'T_att', 'T_succ',
       'Season', 'WDL', 'Team_score', 'Opp_score', 'Team_CS', 'Was_home', 'GW',
       'FPL_name', 'FPL_team', 'FPL_opp_team', 'Name_original', 'FPL_pos',
       'FPL_ast', 'Bonus', 'Bps', 'Player_CS', 'Creativity', 'Element',
       'Fixture', 'Player_GC', 'Goals_scored', 'ICT_index', 'Influence',
       'Kickoff_time', 'FPL_min', 'Own_goals', 'Penalties_missed',
       'Penalties_saved', 'Saves', 'Selected', 'Threat', 'FPL_points',
       'Transfers_balance', 'Transfers_in', 'Transfers_out', 'Price', 'xP',
       'Opp_rating', 'Team_rating', 'Avg_FPL_points'],
      dtype='object')

In [6]:
df_original["Name"].unique().size

454

In [7]:
df.shape

(17927, 74)

In [8]:
df["Season"].unique()

array(['2021-22', '2022-23', '2023-24'], dtype=object)

In [9]:
# df = df[~df["FPL_pos"].isin(["GK"])]

In [10]:
df["Season"].unique()

array(['2021-22', '2022-23', '2023-24'], dtype=object)

In [11]:
df = df[df["Season"].isin(['2021-22', '2022-23'])]

In [12]:
df = df.sort_values(by=["Name", "Date"])

In [13]:
# df = df.reset_index()
# lof = LocalOutlierFactor(n_neighbors=3, contamination=0.01)
# outliers = lof.fit_predict(df.select_dtypes(include=np.number))
# outliers_indices = np.where(outliers == -1)[0]
# df = df.drop(outliers_indices, axis=0)

# Feature selection
Model will use a set of standard features and moving averages for some other feature values. In this case we will use moving averages for last 2, 4 and 30 gameweeks.

In [14]:
# rolling_gameweeks = [2, 4, 30]
rolling_gameweeks = [5]

# to_predict = "FPL_points"
to_predict = "xP"
standard_features = [
             "Was_home", # Home/Away
             # "Team_rating", "Opp_rating", 
             "Rating_difference", # Team ratings
             "Price", # FPl price
             "Transfers_balance",
             # "Transfers_result"
             "Avg_FPL_points"
             ]
features_to_roll_outfield = [
                "Min", 
                # "Start", # time played
                'Gls', 
                # 'Sh', 'SoT', # Goals
                'Ast', # Assists
                # 'CrdY', 'CrdR', # Cards
                "xG", 'xA', # Expected
                'Team_CS', # Defence
                'Team_score', 'Opp_score', 'Team_result', # Team form
                "Player_GC",
                # "xGPoints", "CSPoints", # Position-scaled
                # "Cmp%", "PrgP", "PrgC", "T_succ",
                # 'bonus', 'bps', # Bonus
                # 'ICT_index', # ICT
                "FPL_points", 
                # "xP"
                # "Baseline_points", "Bonus" # FPL points
            ]
features_to_roll_gk = [
                "Min", 
                # "Start", # time played
                # 'Gls', 
                # 'Sh', 'SoT', # Goals
                # 'Ast', # Assists
                # 'CrdY', 'CrdR', # Cards
                # "xG", 'xA', # Expected
                'Team_CS', # Defence
                "Player_GC",
                'Team_score', 'Opp_score', 'Team_result', # Team form
                "Saves",
                # "xGPoints", "CSPoints", # Position-scaled
                # "Cmp%", "PrgP", "PrgC", "T_succ",
                # 'bonus', 'bps', # Bonus
                # 'ICT_index', # ICT
                "FPL_points", 
                # "Baseline_points", "Bonus" # FPL points
            ]
features_to_roll = np.unique([features_to_roll_gk + features_to_roll_outfield])
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "Price", "FPL_pos", "FPL_points"]

In [15]:
features_to_roll

array(['Ast', 'FPL_points', 'Gls', 'Min', 'Opp_score', 'Player_GC',
       'Saves', 'Team_CS', 'Team_result', 'Team_score', 'xA', 'xG'],
      dtype='<U11')

In [16]:
# df[df["Name"] == "Erling-Haaland"]

# Feature engineering

In [17]:
df["Rating_difference"] = df["Team_rating"] / df["Opp_rating"]
df["Baseline_points"] = df["FPL_points"] - df["Bonus"]

In [18]:
def calculate_team_points(row):
    # win - 3 points
    if row['Team_score'] > row['Opp_score']:
        return 3
    # draw - 1 point
    elif row['Team_score'] == row['Opp_score']:
        return 1
    # loss - 0 points
    else:
        return 0
    
df['Team_result'] = df.apply(calculate_team_points, axis=1)

In [19]:
def add_rolling_features(df, standard_features, features_to_roll_outfield, features_to_roll_gk):
    features_outfield = copy.copy(standard_features)
    features_gk = copy.copy(standard_features)
    
    for r in rolling_gameweeks:
        form_means = df.groupby(["Name"])[features_to_roll].rolling(r, min_periods=1).mean().groupby(["Name"]).shift(1).reset_index()
        # print(form_means[form_means["Name"] == "Erling-Haaland"])
        form_means = form_means.fillna(method='bfill') # slightly incorrect, better to drop Nan
        form_means.columns = [f'{col}{"_"}{r}' for col in form_means.columns]
        # features += form_means.columns.tolist()
        features_outfield += [f'{col}{"_"}{r}' for col in features_to_roll_outfield]
        features_gk += [f'{col}{"_"}{r}' for col in features_to_roll_gk]
        # features = list(filter(lambda x: x not in ["Name_" + str(r)], features))
        df = pd.concat([df.reset_index(), form_means], axis=1)
        # df = df.merge(form_means, left_index=True, right_index=True)
        df = df.drop([col for col in df.columns if col.startswith('level')], axis=1)
        
    return df.reset_index(), features_outfield, features_gk

In [20]:
def ohe(df, ohe_columns, features_outfield, features_gk):
    # one hot encoding
    for c in ohe_columns:
        ohe_c = pd.get_dummies(df[c], dtype="int64")
        df = pd.concat([df, ohe_c], axis=1)

        features_outfield += ohe_c.columns.tolist()
        features_gk += ohe_c.columns.tolist()
        
    return df, features_outfield, features_gk

In [21]:
def label_encoding(df, column_to_encode):
    
    mapping_dict = {
        'Y': 1, # Starting eleven
        'Y*': 1, # Starting eleven as captain
        'N': 0, # Not in starting eleven
    }
    
    df[column_to_encode] = df[column_to_encode].map(mapping_dict)
    
    return df

In [22]:
df = label_encoding(df, "Start")

In [23]:
df, features_outfield, features_gk = add_rolling_features(df, standard_features, features_to_roll_outfield, features_to_roll_gk)

In [24]:
features_gk

['Was_home',
 'Rating_difference',
 'Price',
 'Transfers_balance',
 'Avg_FPL_points',
 'Min_5',
 'Team_CS_5',
 'Player_GC_5',
 'Team_score_5',
 'Opp_score_5',
 'Team_result_5',
 'Saves_5',
 'FPL_points_5']

In [25]:
features_outfield

['Was_home',
 'Rating_difference',
 'Price',
 'Transfers_balance',
 'Avg_FPL_points',
 'Min_5',
 'Gls_5',
 'Ast_5',
 'xG_5',
 'xA_5',
 'Team_CS_5',
 'Team_score_5',
 'Opp_score_5',
 'Team_result_5',
 'Player_GC_5',
 'FPL_points_5']

In [26]:
df, features_outfield, features_gk = ohe(df, ["FPL_pos"], features_outfield, features_gk)

In [27]:
df.columns

Index(['level_0', 'index', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA', 'SCA',
       'GCA', 'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries', 'PrgC', 'T_att',
       'T_succ', 'Season', 'WDL', 'Team_score', 'Opp_score', 'Team_CS',
       'Was_home', 'GW', 'FPL_name', 'FPL_team', 'FPL_opp_team',
       'Name_original', 'FPL_pos', 'FPL_ast', 'Bonus', 'Bps', 'Player_CS',
       'Creativity', 'Element', 'Fixture', 'Player_GC', 'Goals_scored',
       'ICT_index', 'Influence', 'Kickoff_time', 'FPL_min', 'Own_goals',
       'Penalties_missed', 'Penalties_saved', 'Saves', 'Selected', 'Threat',
       'FPL_points', 'Transfers_balance', 'Transfers_in', 'Transfers_out',
       'Price', 'xP', 'Opp_rating', 'Team_rating', 'Avg_FPL_points',
       'Rating_difference', 'Baseline_points', 'Team_result', 'Name_5',
       'Ast_5', 'FPL_points_5', 'Gls_5', '

In [28]:
# dropping unwanted columns
# features = [col for col in features if not col.startswith('level')]
df = df[np.unique(features_outfield + features_gk + info + [to_predict])]

In [29]:
df = df[pd.to_numeric(df["GW"], errors="coerce").notna()]
df["GW"] = df["GW"].astype("uint64")

In [30]:
# dropping NaNs
df = df.dropna(axis=0)

In [31]:
# df = df.set_index(["Name_original", "Season", "GW"], drop=False)

In [32]:
df.columns

Index(['Ast_5', 'Avg_FPL_points', 'DEF', 'FPL_points', 'FPL_points_5',
       'FPL_pos', 'FWD', 'GK', 'GW', 'Gls_5', 'MID', 'Min_5', 'Name_original',
       'Opp_rating', 'Opp_score_5', 'Opponent', 'Player_GC_5', 'Price',
       'Rating_difference', 'Saves_5', 'Season', 'Team', 'Team_CS_5',
       'Team_rating', 'Team_result_5', 'Team_score_5', 'Transfers_balance',
       'Was_home', 'xA_5', 'xG_5', 'xP'],
      dtype='object')

In [33]:
df.shape

(17422, 31)

In [34]:
# df = df[~df["FPL_pos"].isin(["GK"])] # only outfield players

# Data split into train and valid
Let's take entire 2021-22 season and 30 first gameweeks of 2022-23 season as training data and 8 last gameweeks of that season as valid data (~10% of all rows).

In [35]:
def shrink_df_to_top_players(df, n_players, min_fixtures):
    # Calculate average scores
    average_scores = df.groupby('Name_original')[to_predict].mean()
    
    # Calculate player counts
    player_counts = df['Name_original'].value_counts()
    
    # Filter out players with less than min_fixtures occurrences
    popular_players = player_counts[player_counts >= min_fixtures].index
    
    print("Unique players with min_fixtures:", popular_players.size)

    # Sort players by average scores
    sorted_players = average_scores.sort_values(ascending=False)

    # Get the top n_players players
    top_popular_players = sorted_players[sorted_players.index.isin(popular_players)].head(n_players)

    # Filter the original DataFrame
    top_players_df = df[df['Name_original'].isin(top_popular_players.index)]
    
    print("Unique players left:", top_players_df['Name_original'].unique().size)
    
    return top_players_df

In [36]:
CUT_OFF_GAMEWEEK = 31
SEASON_TO_PREDICT = "2022-23"

In [37]:
df_train = shrink_df_to_top_players(df, 550, 10)
# df_train["Name_original"].unique()

Unique players with min_fixtures: 385
Unique players left: 385


In [38]:
df = df.set_index(["Name_original", "Season", "GW"], drop=False)

In [39]:
# training data
X_train = df_train[((df_train["Season"] < SEASON_TO_PREDICT) | (df_train["GW"] <= CUT_OFF_GAMEWEEK - 1))].reset_index(drop=True)
y_train = df_train[((df_train["Season"] < SEASON_TO_PREDICT) | (df_train["GW"] <= CUT_OFF_GAMEWEEK - 1))].reset_index(drop=True)

In [40]:
# only 31st gameweek
X_test = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) ].reset_index(drop=True)
y_test = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) ].reset_index(drop=True)

In [41]:
# all remaining gameweeks
X_test_all_remaining = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) ].reset_index(drop=True)
y_test_all_remaining = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) ].reset_index(drop=True)

In [42]:
X_train.shape, X_test.shape, X_test_all_remaining.shape

((15187, 31), (264, 31), (2170, 31))

# Model

In [43]:
# model = GradientBoostingRegressor(random_state=42)
model = PositionalModel(features_gk, features_outfield, to_predict)
# model = RandomForestRegressor(random_state=42)

In [44]:
%%time
model.fit(
    X_train,
    y_train,
    X_test_all_remaining,
    y_test_all_remaining
)

CPU times: total: 5.83 s
Wall time: 834 ms


# Getting predictions

In [45]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    
    if all_remaining:
        df_predictions = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT)].reset_index(drop=True)
    else:
        df_predictions = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT)].reset_index(drop=True)
    
    df_predictions.loc[:, "Pred"] = predictions
    
    preds = df_predictions[info + [to_predict] + ["Pred"]].sort_values(by=["Pred"], ascending = False)
    
    return preds, predictions

In [46]:
X_test.columns

Index(['Ast_5', 'Avg_FPL_points', 'DEF', 'FPL_points', 'FPL_points_5',
       'FPL_pos', 'FWD', 'GK', 'GW', 'Gls_5', 'MID', 'Min_5', 'Name_original',
       'Opp_rating', 'Opp_score_5', 'Opponent', 'Player_GC_5', 'Price',
       'Rating_difference', 'Saves_5', 'Season', 'Team', 'Team_CS_5',
       'Team_rating', 'Team_result_5', 'Team_score_5', 'Transfers_balance',
       'Was_home', 'xA_5', 'xG_5', 'xP'],
      dtype='object')

In [47]:
preds, predictions = get_predictions(model, df, X_test)

In [48]:
preds_all_gameweeks, predictions_all = get_predictions(model, df, X_test_all_remaining, True)

In [49]:
df.columns

Index(['Ast_5', 'Avg_FPL_points', 'DEF', 'FPL_points', 'FPL_points_5',
       'FPL_pos', 'FWD', 'GK', 'GW', 'Gls_5', 'MID', 'Min_5', 'Name_original',
       'Opp_rating', 'Opp_score_5', 'Opponent', 'Player_GC_5', 'Price',
       'Rating_difference', 'Saves_5', 'Season', 'Team', 'Team_CS_5',
       'Team_rating', 'Team_result_5', 'Team_score_5', 'Transfers_balance',
       'Was_home', 'xA_5', 'xG_5', 'xP'],
      dtype='object')

# Custom metric
Such models are usually used to compare players and decide who to pick for upcoming gameweek(s). For each player pair in a subset of players model can either predict the outcome well (e.g. player A scores more than player B and model predicts exactly that) or predict wrongly (e.g. player A scores more than player B but model predicts player B > player A). Pairwise_accuracy returns the percentage of corrected predicted pairs. Pairwise_accuracy_topX is a variation of this metric calculated only for X highest scoring players of last two seasons.

In [50]:
def get_top_performer_names(df, no_top, no_gws):
    # takes no_top players that recorded highest average FPL points in no_gws last gameweeks
    return df.reset_index(drop=True).groupby("Name_original")[to_predict].mean().groupby("Name_original").tail(no_gws).sort_values(ascending=False).head(no_top).index.to_list()

In [51]:
def pairwise_accuracy(true_scores, predicted_scores):
    if len(predicted_scores) != len(true_scores):
        raise ValueError("The length of predicted_scores and true_scores must be the same.")

    num_pairs = 0
    num_correct_pairs = 0

    for i in range(len(predicted_scores)):
        for j in range(i + 1, len(predicted_scores)):
            # print(i)
            # print(predicted_scores)
            # print(predicted_scores[i])
            # Check if the predicted order matches the true order
            if (predicted_scores[i] > predicted_scores[j] and true_scores[i] >= true_scores[j]) or \
               (predicted_scores[i] < predicted_scores[j] and true_scores[i] <= true_scores[j]):
                num_correct_pairs += 1
            num_pairs += 1

    pairwise_accuracy = num_correct_pairs / num_pairs
    return pairwise_accuracy

In [52]:
def get_top_performers(df, top_x, all_gw=False):
    if all_gw:
        top_performers = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name_original"].isin(get_top_performer_names(df, top_x, 50)))]
    else:
        top_performers = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name_original"].isin(get_top_performer_names(df, top_x, 50)))]

    X = top_performers.reset_index(drop=True)
    y = np.array(top_performers[to_predict].reset_index(drop=True))
    
    return X, y

In [53]:
def pairwise_accuracy_topX(model, df, top_x, all_gw=False):
    # pairwise_accuracy for top_X players
    if all_gw:
        top_performers = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name_original"].isin(get_top_performer_names(df, top_x, 50)))]
    else:
        top_performers = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name_original"].isin(get_top_performer_names(df, top_x, 50)))]

    X = top_performers.reset_index(drop=True)
    y = np.array(top_performers[to_predict].reset_index(drop=True))
    preds = model.predict(X)
    
    return  pairwise_accuracy(y, preds)

In [54]:
def mae_topX(model, df, top_x, all_gw=False):
    X, y = get_top_performers(df, top_x, all_gw)
    preds = model.predict(X)
    
    return mean_absolute_error(y, preds)

In [55]:
def mse_topX(model, df, top_x, all_gw=False):
    X, y = get_top_performers(df, top_x, all_gw)
    preds = model.predict(X)
    
    return mean_squared_error(y, preds)

# Evaluation

In [56]:
def evaluate(model, df, predictions, y_true, all_gw=False):
    # function to calculate different metrics for a given model
    mae = mean_absolute_error(y_true["FPL_points"], predictions)
    mae_top100 = mae_topX(model, df, 100, all_gw)
    mae_top20 = mae_topX(model, df, 20, all_gw)
    
    mse = mean_squared_error(y_true["FPL_points"], predictions)
    mse_top100 = mse_topX(model, df, 100, all_gw)
    mse_top20 = mse_topX(model, df, 20, all_gw)

    pairwise_acc = pairwise_accuracy(y_true["FPL_points"], predictions)
    pairwise_accuracy_top100 = pairwise_accuracy_topX(model, df, 100, all_gw)
    pairwise_accuracy_top20 = pairwise_accuracy_topX(model, df, 20, all_gw)
    
    return pd.DataFrame( {'MAE': [mae, mae_top100, mae_top20], 
                          'MSE': [mse, mse_top100, mse_top20], 
                          'PWA': [pairwise_acc, pairwise_accuracy_top100, pairwise_accuracy_top20]}, 
                        index=["All", "Top100", "Top20"])

In [57]:
# next gameweek
evaluate(model, df, predictions, y_test, all_gw=False)

,MAE,MSE,PWA
All,1.916239,8.225300,0.796319
Top100,2.076553,7.236023,0.613464
Top20,2.471294,8.829022,0.585905


In [58]:
# all remaining gameweeks
evaluate(model, df, predictions_all, y_test_all_remaining, all_gw=True)

,MAE,MSE,PWA
All,1.910745,7.679888,0.757337
Top100,2.050655,7.122862,0.644138
Top20,1.986547,4.897188,0.777778


In [59]:
pred_sum = preds_all_gameweeks["Pred"].sum()
fpl_sum = preds_all_gameweeks["FPL_points"].sum()
xp_sum = preds_all_gameweeks["xP"].sum()

pd.DataFrame([pred_sum, fpl_sum, xp_sum], index=["Pred", "FPL", "xP"], columns=["Sum"])

,Sum
Pred,5290.975474
FPL,5242.000000
xP,5640.700000


# Predictions - next gameweek only

In [ ]:
preds.head(30)

In [ ]:
preds[preds["FPL_pos"] == "GK"].head(20)

In [ ]:
preds[preds["Team"] == "Manchester City"].head(20)

# Predictions - all remaining gameweeks

In [ ]:
preds_all_gameweeks.head(15)

In [ ]:
preds_all_gameweeks[preds_all_gameweeks["Name_original"] == "Mohamed Salah"].sort_values(by=["GW"])

In [ ]:
preds_all_gameweeks[preds_all_gameweeks["Name_original"] == "Erling Haaland"].sort_values(by=["GW"])

# Saving to file

In [ ]:
preds_all_gameweeks.to_csv("predictions/all_predictions.csv", index=False)

In [ ]:
pickle.dump(model, open("models/GBR_positional.pkl", 'wb'))

# Feature importance and influence

In [ ]:
# explainer = shap.Explainer(model.predict, X_test_all_remaining)
# shap_values = explainer(X_test_all_remaining)

In [ ]:
# shap.plots.bar(shap_values, max_display=15)

In [ ]:
# shap.plots.beeswarm(shap_values, max_display=15)

In [ ]:
# # explaining Erling Haaland`s score in gameweek 31
# shap.plots.bar(shap_values[69], max_display=15)

In [ ]:
# # Salah vs Nottingham Forest
# shap.plots.waterfall(shap_values[1396])